In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [2]:
ls

depression_submissions.zst
happy_submissions.parquet
happy_submissions.zst
reddit_data_preprocessing_Positivity.ipynb
reddit_data_preprocessing_happy.ipynb
reddit_data_preprocessing_v1.ipynb


In [ ]:
## step1. load downloaded submission reddit dump 

In [13]:
positivity_submissions =\
    pd.read_json('Positivity_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 4000000)

In [14]:
# convert the timestamp to utc
positivity_submissions['created_utc'] =\
    positivity_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [15]:
## step2. download the file into parquet file to speed up data load later stage

In [16]:
positivity_submissions.to_parquet('positivity_submissions.parquet', engine='pyarrow')

In [17]:
## step3. read as parquet file 

In [4]:
positivity_submissions = pd.read_parquet('positivity_submissions.parquet',engine='pyarrow')
positivity_submissions

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,retrieved_on,rte_mode,score,secure_media,secure_media_embed,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status,downs,num_reports,banned_by,name,likes,clicked,saved,ups,approved_by,selftext_html,created,report_reasons,user_reports,mod_reports,from,quarantine,from_kind,from_id,preview,post_hint,view_count,approved_at_utc,banned_at_utc,can_mod_post,pinned,mod_note,mod_reason_by,mod_reason_title,crosspost_parent,crosspost_parent_list,subreddit_subscribers,author_flair_template_id,category,content_categories,is_original_content,media_only,post_categories,pwls,removal_reason,visited,wls,media_metadata,author_created_utc,author_fullname,is_meta,link_flair_background_color,link_flair_template_id,author_patreon_flair,gildings,is_robot_indexable,author_cakeday,all_awardings,total_awards_received,allow_live_comments,discussion_type,author_premium,awarders,removed_by,removed_by_category,treatment_tags,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,poll_data,gallery_data,is_gallery,call_to_action
0,1.0,Drewbacca,None,None,[],None,None,text,0.0,1.0,0.0,2010-10-18 21:17:49,None,en.wikipedia.org,0,0.0,0.0,0.0,dt01w,1.0,0.0,False,0.0,NaN,[],NaN,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,0,0.0,False,None,/r/Positivity/comments/dt01w/optimism/,1.522964e+09,markdown,2,None,"{'content': None, 'height': None, 'media_domai...",,1.0,0.0,0.0,Positivity,t5_2s5f5,r/Positivity,public,NaN,default,NaN,NaN,Optimism,http://en.wikipedia.org/wiki/Optimism,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None
1,NaN,swefpelego,None,None,None,None,None,None,NaN,NaN,NaN,2012-03-04 07:03:18,None,self.Positivity,0,NaN,0.0,NaN,qgxmd,NaN,NaN,True,NaN,NaN,None,NaN,None,None,NaN,None,"{'content': None, 'height': None, 'media_domai...",NaN,0,NaN,False,None,/r/Positivity/comments/qgxmd/why_hellloooooooo...,NaN,None,1,None,None,Just hope you're having a good day and that yo...,NaN,NaN,NaN,Positivity,t5_2s5f5,None,None,NaN,default,NaN,NaN,Why hellloooooooooooo there!,http://www.reddit.com/r/Positivity/comments/qg...,None,5.0,NaN,NaN,t3_qgxmd,NaN,0.0,0.0,6.0,NaN,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",1.330845e+09,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,None
2,NaN,goodsignguy,None,None,None,None,None,None,NaN,NaN,NaN,2012-04-14 15:21:16,None,youtube.com,0,NaN,0.0,NaN,s9jfb,NaN,NaN,False,NaN,NaN,None,NaN,None,None,NaN,"{'oembed': {'author_name': 'ThisIsAGoodSign', ...","{'content': '&lt;iframe width=""600"" height=""33...",NaN,1,NaN,False,None,/r/Positivity/comments/s9jfb/this_is_a_good_si...,NaN,None,1,None,None,,NaN,NaN,NaN,Positivity,t5_2s5f5,None,None,NaN,default,NaN,NaN,This is a Good Sign! \nCBS news coverage of ou...,http://www.youtube.com/watch?v=1FDh6zL7kRA&amp...,None,0.0,NaN,NaN,t3_s9jfb,NaN,0.0,0.0,1.0,NaN,None,1.334420e+09,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None

In [5]:
## step4. feature selection : select only necessary columns

In [6]:
positivity_submissions =\
    positivity_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
positivity_submissions


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,Positivity,Drewbacca,False,NaN,Optimism,,http://en.wikipedia.org/wiki/Optimism,2010-10-18 21:17:49
1,Positivity,swefpelego,False,NaN,Why hellloooooooooooo there!,Just hope you're having a good day and that yo...,http://www.reddit.com/r/Positivity/comments/qg...,2012-03-04 07:03:18
2,Positivity,goodsignguy,False,NaN,This is a Good Sign! \nCBS news coverage of ou...,,http://www.youtube.com/watch?v=1FDh6zL7kRA&amp...,2012-04-14 15:21:16
3,Positivity,thepathyny,False,NaN,"Regaining balance in a workcentric world. ""Cha...",,http://thepathny.wordpress.com/2012/05/31/work...,2012-06-21 16:13:49
4,Positivity,dpquiltsca,False,NaN,What NOT to do with Energy Zappers.,,http://daleannepotter.com/energy-zappers/,2012-09-25 16:50:38
...,...,...,...,...,...,...,...,...
18990,Positivity,TyMeditation,False,NaN,12 Positive Mindset for 12 Months,,https://youtube.com/watch?v=5Ozw0H1HhxU&amp;fe...,2022-12-31 17:01:31
18991,Positivity,classicspoonbill,False,NaN,App recommendations,Hi I’m trying to be more positive and work on ...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 17:03:08
18992,Positivity,coding_beast_404,False,NaN,What are your positive affirmations for this n...,,https://i.redd.it/9vdlvloet99a1.jpg,2022-12-31 17:44:20
18993,Positivity,plettpaul,False,NaN,Priorities,"When I’m feeling overwhelmed, I like to take a...",https://youtu.be/7Rd4PMIYj9Q,2022-12-31 17:50:38


In [7]:
## step5. basic pre-processing 

In [8]:
# remove blank body posts
positivity_submissions = positivity_submissions[positivity_submissions['selftext'].apply(lambda x : len(x)!=0)]
positivity_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
1,Positivity,swefpelego,False,NaN,Why hellloooooooooooo there!,Just hope you're having a good day and that yo...,http://www.reddit.com/r/Positivity/comments/qg...,2012-03-04 07:03:18
43,Positivity,DoctorCharm,False,NaN,An eulogy for a homeless man in Boston,"This was my status update for the day on FB, a...",http://www.reddit.com/r/Positivity/comments/1d...,2013-05-05 11:18:24
47,Positivity,perouzap,False,NaN,Think Positive Campaign,Join us in making positivity the reigning emot...,http://www.reddit.com/r/Positivity/comments/1l...,2013-09-07 07:46:38
56,Positivity,PositiveLiving1998,False,NaN,Positive Thinking,"Think positive, live your life, and do what yo...",http://www.reddit.com/r/Positivity/comments/1s...,2013-12-15 02:15:10
61,Positivity,Sharoncavill,False,NaN,Everything Is Happening For Me,Everything is happening FOR me is a powerful c...,http://www.reddit.com/r/Positivity/comments/1u...,2014-01-06 23:37:58
...,...,...,...,...,...,...,...,...
18984,Positivity,BeHappyAndWealthy,False,NaN,another year another sparkle,SO we are landing in 2023 tomorrow ( India) an...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 07:00:07
18988,Positivity,[deleted],False,NaN,2022 is finally over! Come 2023 you beautiful!,[deleted],,2022-12-31 15:40:32
18989,Positivity,[deleted],False,NaN,:DD !!!,[deleted],,2022-12-31 16:45:07
18991,Positivity,classicspoonbill,False,NaN,App recommendations,Hi I’m trying to be more positive and work on ...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 17:03:08


In [9]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
positivity_submissions[positivity_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
295,Positivity,[deleted],False,NaN,Save for YOU not for goals!,[deleted],https://medium.com/@TipYourself/save-for-you-n...,2015-09-03 23:19:18
296,Positivity,bigced,False,NaN,Practice Positive Self-Talk! - Daily Word,[removed],https://www.reddit.com/r/Positivity/comments/3...,2015-09-03 23:27:29
314,Positivity,[deleted],False,NaN,"It would be awesome, if you guys (teenagers) w...",[deleted],https://docs.google.com/spreadsheets/d/1wPGmNt...,2015-10-24 23:56:59
341,Positivity,[deleted],False,NaN,40 Ways to Practice Self-Kindness.,[deleted],http://kindnessblog.com/2014/01/29/40-ways-to-...,2015-12-17 11:59:35
343,Positivity,[deleted],False,NaN,8 Things that make you Different and Positive ...,[deleted],https://www.quora.com/What-are-some-best-easy-...,2015-12-19 06:03:05
...,...,...,...,...,...,...,...,...
18965,Positivity,[deleted],False,NaN,Hi everyone. I hope you all crush it and lead ...,[deleted],,2022-12-29 04:46:02
18972,Positivity,[deleted],False,NaN,The real reason you don’t love yourself.❤️NO BS!,[removed],,2022-12-29 14:17:15
18982,Positivity,[deleted],False,NaN,2022 Rewind: The 10 best videos in 4K Ultra HD...,[deleted],,2022-12-30 14:30:23
18988,Positivity,[deleted],False,NaN,2022 is finally over! Come 2023 you beautiful!,[deleted],,2022-12-31 15:40:32


In [10]:
positivity_submissions = positivity_submissions[~positivity_submissions['selftext'].isin(opt_out_list)]
print(len(positivity_submissions))
positivity_submissions

2968


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
1,Positivity,swefpelego,False,NaN,Why hellloooooooooooo there!,Just hope you're having a good day and that yo...,http://www.reddit.com/r/Positivity/comments/qg...,2012-03-04 07:03:18
43,Positivity,DoctorCharm,False,NaN,An eulogy for a homeless man in Boston,"This was my status update for the day on FB, a...",http://www.reddit.com/r/Positivity/comments/1d...,2013-05-05 11:18:24
47,Positivity,perouzap,False,NaN,Think Positive Campaign,Join us in making positivity the reigning emot...,http://www.reddit.com/r/Positivity/comments/1l...,2013-09-07 07:46:38
56,Positivity,PositiveLiving1998,False,NaN,Positive Thinking,"Think positive, live your life, and do what yo...",http://www.reddit.com/r/Positivity/comments/1s...,2013-12-15 02:15:10
61,Positivity,Sharoncavill,False,NaN,Everything Is Happening For Me,Everything is happening FOR me is a powerful c...,http://www.reddit.com/r/Positivity/comments/1u...,2014-01-06 23:37:58
...,...,...,...,...,...,...,...,...
18956,Positivity,simpingforMinYoongi,False,NaN,Reminder:,Your value is not what you can do for others o...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-28 14:19:07
18962,Positivity,letsstartanew2,False,NaN,The year finally ends. Me:,"Come 2023, you beautiful!",https://youtu.be/JdE6tS2awJk,2022-12-29 02:35:35
18984,Positivity,BeHappyAndWealthy,False,NaN,another year another sparkle,SO we are landing in 2023 tomorrow ( India) an...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 07:00:07
18991,Positivity,classicspoonbill,False,NaN,App recommendations,Hi I’m trying to be more positive and work on ...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 17:03:08


In [11]:
# step6. final data quality check 

In [12]:
print(f'Author : {positivity_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {positivity_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {positivity_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {positivity_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {positivity_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {positivity_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {positivity_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {positivity_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {positivity_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {positivity_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {positivity_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {positivity_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : swefpelego
Subreddit : Positivity
Teenager(True/False) : False
Posted : 2012-03-04 07:03:18

Title : Why hellloooooooooooo there!
Body : Just hope you're having a good day and that you've been well. It's not so cold today and was actually kind of nice considering it's March. 

All in all it's all been good! Take it easy. 



Author : DoctorCharm
Subreddit : Positivity
Teenager(True/False) : False
Posted : 2013-05-05 11:18:24

Title : An eulogy for a homeless man in Boston
Body : This was my status update for the day on FB, and my very first reddit post. I do not want to start my first reddit post to be a sad one, but I strongly believe that people should know what is happening out there among the homeless community. Without further ado:

An Eulogy For My Homeless Friend, "Da-Man"

There was a homeless man that I would treat out to lunch every other week. His name was, "Da-Man". He is also known as the homeless man who can do pushups with one hand. He used to hang out right by 

In [13]:
# step7. save final pre-processed data into parquet
positivity_submissions.to_parquet('positivity_submissions_preprocessed.parquet', engine='pyarrow')
positivity_submissions_preprocessed = pd.read_parquet('positivity_submissions_preprocessed.parquet',engine='pyarrow')
positivity_submissions_preprocessed


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
1,Positivity,swefpelego,False,NaN,Why hellloooooooooooo there!,Just hope you're having a good day and that yo...,http://www.reddit.com/r/Positivity/comments/qg...,2012-03-04 07:03:18
43,Positivity,DoctorCharm,False,NaN,An eulogy for a homeless man in Boston,"This was my status update for the day on FB, a...",http://www.reddit.com/r/Positivity/comments/1d...,2013-05-05 11:18:24
47,Positivity,perouzap,False,NaN,Think Positive Campaign,Join us in making positivity the reigning emot...,http://www.reddit.com/r/Positivity/comments/1l...,2013-09-07 07:46:38
56,Positivity,PositiveLiving1998,False,NaN,Positive Thinking,"Think positive, live your life, and do what yo...",http://www.reddit.com/r/Positivity/comments/1s...,2013-12-15 02:15:10
61,Positivity,Sharoncavill,False,NaN,Everything Is Happening For Me,Everything is happening FOR me is a powerful c...,http://www.reddit.com/r/Positivity/comments/1u...,2014-01-06 23:37:58
...,...,...,...,...,...,...,...,...
18956,Positivity,simpingforMinYoongi,False,NaN,Reminder:,Your value is not what you can do for others o...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-28 14:19:07
18962,Positivity,letsstartanew2,False,NaN,The year finally ends. Me:,"Come 2023, you beautiful!",https://youtu.be/JdE6tS2awJk,2022-12-29 02:35:35
18984,Positivity,BeHappyAndWealthy,False,NaN,another year another sparkle,SO we are landing in 2023 tomorrow ( India) an...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 07:00:07
18991,Positivity,classicspoonbill,False,NaN,App recommendations,Hi I’m trying to be more positive and work on ...,https://www.reddit.com/r/Positivity/comments/z...,2022-12-31 17:03:08
